In [1]:
import torch
from models.naive_cnn import NaiveConvolutionNetwork
from models.densenet import DenseNet
from models.resnet import ResNet
import numpy as np

from utils import load_dataset, evaluate, compute_AUCs

In [2]:
PATH = 'saved_models/DenseNet.pt'

model = torch.load(PATH)
model.eval()

NaiveConvolutionNetwork(
  (layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ConvolutionBlock(
      (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (batchnorm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pooling): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    )
    (2): ConvolutionBlock(
      (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (batchnorm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pooling): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    )
    (3): ConvolutionBlock(
      (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (batchnorm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (acti

In [3]:
_, _, test_loader = load_dataset(64)

LOADING DATASET


In [4]:
pos_weight = torch.tensor([9.719982661465107,
                                 40.447330447330444,
                                 8.425640640264522,
                                 5.6423934376729905,
                                 19.512704490080054,
                                 17.73208919816543,
                                 82.86770140428678,
                                 21.162702906757268,
                                 24.023998285836726,
                                 48.68432479374729,
                                 44.56279809220986,
                                 66.5005931198102,
                                 33.122599704579024,
                                 493.92070484581495])

criterion =  torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
device = 'cuda'

eval_dict  = evaluate(model, test_loader, criterion, device)

----- Evaluating ------


100%|████████████████████████████████████████████████████████████████████████████████| 176/176 [02:43<00:00,  1.08it/s]


In [5]:
N_CLASSES = 14
CLASS_NAMES = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
               'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia']

print(f"Eval loss is:{eval_dict['loss']}")
print(f"Eval accuracy is:{eval_dict['acc']}")
print(f"Eval f1 score is:{eval_dict['f1']}")

AUROCs = compute_AUCs(eval_dict['labels'], eval_dict['logits'])
AUROC_avg = np.array(AUROCs).mean()
print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
for i in range(N_CLASSES):
    print('The AUROC of {} is {}'.format(CLASS_NAMES[i], AUROCs[i]))

Eval loss is:1.2192518260668626
Eval accuracy is:0.13239424062764676
Eval f1 score is:[0.27106292 0.09976526 0.36022119 0.34883561 0.12837433 0.12412928
 0.0362962  0.15114773 0.16105709 0.11631095 0.06873614 0.05191473
 0.09414048 0.00953406]
The average AUROC is 0.699
The AUROC of Atelectasis is 0.6869357954519645
The AUROC of Cardiomegaly is 0.7375243389271386
The AUROC of Effusion is 0.7680720923029584
The AUROC of Infiltration is 0.6444424192721708
The AUROC of Mass is 0.6175235052563098
The AUROC of Nodule is 0.5701688712883661
The AUROC of Pneumonia is 0.6673428025880495
The AUROC of Pneumothorax is 0.6802118912995292
The AUROC of Consolidation is 0.75299415663238
The AUROC of Edema is 0.8351334248196116
The AUROC of Emphysema is 0.664975882034347
The AUROC of Fibrosis is 0.6951509580707766
The AUROC of Pleural_Thickening is 0.6527114284576958
The AUROC of Hernia is 0.8077522644089569
